In [1]:
import pandas as pd
import requests
import math
import time
from tqdm import tqdm
import os
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import xmltodict as saveme
from tqdm import tqdm_notebook

In [85]:
df = pd.read_csv('/Users/corey/Documents/GitHub/WaterSentiment/data/annual_csvs/scopus_search_year_r2_1991.csv')
len(df['eid'].unique())

1988

In [84]:
1.9e5

190000.0

In [386]:
wkdir = os.path.dirname(os.getcwd())

data_dir  = wkdir + '/data'

search_results = pd.read_csv(data_dir+'/scopus_search_results.csv', low_memory = False)

search_results['date'] = pd.to_datetime(search_results['prism:coverDate'], format='%Y-%m-%d', errors='coerce')

search_results = search_results[search_results.date.notna()]
# search_results = search_results.set_index('date')

api_key = open(wkdir+'/code/keys.gitignore', "r").readlines()

print('Master dataframe of search results imported')

Master dataframe of search results imported


In [383]:
print('Beginning of song to retrieve abstracts:')

#make an empty list to store failures
failed_pages = []

# create master dataframe with all the first article in the list

idx = search_results.iloc[0]['dc:identifier'].split(':')[1]
response = requests.get('https://api.elsevier.com/content/abstract/scopus_id/'+idx+'?view=META_ABS&apikey='+ api_key[2])

abstracts = pd.json_normalize(saveme.parse(response.text))


for i, a in tqdm(search_results[1:].iterrows()):
    try:
        #print("Working on article " + str(i) +' of'  + str(len(search_results)))
        #Get scupis ID:
        idx = a['dc:identifier'].split(':')[1]

        #Format URL to retrieve the abstract:
        response = requests.get('https://api.elsevier.com/content/abstract/scopus_id/'+idx+'?view=META_ABS&apikey='+ api_key[2])
        abstract = pd.json_normalize(saveme.parse(response.text))
        abstracts = abstracts.append(abstract)
    except:
        failed_pages = failed_pages.append(idx)
        continue
    time.sleep(0.12)
    
abstracts.to_csv(wkdir+'/data/abstracts_full.csv')

print('Failed abstract retrievals: ' + str(len(failed_pages)), "\n", failed_pages)

print("Ding, end of song!")

Beginning of song to retrieve abstracts:





0it [00:00, ?it/s]


1it [00:00,  2.72it/s]

Failed abstract retrievals : 0 
 []
Ding, end of song!


In [396]:
'https://api.elsevier.com/content/abstract/scopus_id/'+idx+'?view=META_ABS&apikey='+ api_key[2]
# dates = []
# cals = []

# for t in search_results.iterrows():
#     try:
#         #print(t)
#         dates.append(t[0])
#         title = t[1][7]
#         cals.append("california" in title.lower())
#     except:
#         continue

'https://api.elsevier.com/content/abstract/scopus_id/84865404754?view=META_ABS&apikey=19ab7c717adf69e87207438e0624dade'

In [ ]:
# cali = pd.DataFrame({'dates': dates, 'calis': cals})
# cali = cali.set_index('dates')

# cali.sum()

In [384]:
# grps = pd.DataFrame(search_results.eid.groupby(search_results.index.year).count())

# grps['calis'] = yrs['calis']

# ax = grps.plot(x=grps.index, kind='bar')

# # Make most of the ticklabels empty so the labels don't get too crowded
# ticklabels = ['']*len(search_results.index)
# # Every 4th ticklable shows the month and day
# #ticklabels[::4] = [item.strftime('%b %d') for item in search_results.index[::4]]
# # Every 12th ticklabel includes the year
# ticklabels[::12] = [item.strftime('%b %d\n%Y') for item in search_results.index[::12]]
# ax.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))
# ax.set_ylabel('article count')
# ax.set_title('"ground water" OR groundwater OR ground-water OR aquifer')
# plt.tight_layout()
# plt.savefig('article_counts.png', dpi=300)